In [1]:
import pandas as pd
import nose.tools
import numpy as np

# Data Tidying and Cleaning Lab
## Reading, tidying and cleaning data. Preparing data for exploration, mining, analysis and learning

### Problem 1. Read the dataset (2 points)
The dataset [here](http://archive.ics.uci.edu/ml/datasets/Auto+MPG) contains information about fuel consumption in different cars.

Click the "Data Folder" link and read `auto_mpg.data` into Python. You can download it, if you wish, or you can read it directly from the link.

Give meaningful (and "Pythonic") column names, as per the `auto_mpg.names` file:
1. mpg
2. cylinders
3. displacement
4. horsepower
5. weight
6. acceleration
7. model_year
8. origin
9. car_name

In [2]:
# Read data - there are no headers in the provided data set
mpg_data = pd.read_fwf("data/auto-mpg.data", header=None)
# Assign column (features) headers
mpg_data.columns=["mpg", "cylinders", "displacement", "horsepower", "weight", "aceleration", "model_year", "origin", "car_name"]

In [3]:
nose.tools.assert_is_not_none(mpg_data)

Print the first 4 rows in the dataset to get a feel of what it looks like:

In [4]:
# Show firs 4 rows in the data set
mpg_data.head(4)

,mpg,cylinders,displacement,horsepower,weight,aceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,"""buick skylark 320"""
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,"""plymouth satellite"""
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,"""amc rebel sst"""


### Problem 2. Inspect the dataset (1 point)
Write a function which accepts a dataset and returns the number of observations and features in it, like so: 

``` 10 observations on 15 features```

Where 10 and 15 should be replaced with the real numbers. Test your function with the `auto_mpg` dataset.

Make sure the function works with other datasets (don't worry about "1 features" or "1 observations", just leave it as it is).

In [5]:
def observations_and_features(dataset):
    """
    Returns the number of observations and features in the provided dataset
    """
    observations = dataset.shape[0]
    features = dataset.shape[1]
    result = f"{observations} observations on {features} features"
    return result

In [6]:
print(observations_and_features(mpg_data))

398 observations on 9 features


Inspect the data types for each column.

In [7]:
# Show all column datatypes
mpg_data.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
aceleration     float64
model_year        int64
origin            int64
car_name         object
dtype: object

### Problem 3. Correct errors (1 point)
The `horsepower` column looks strange. It's a string but it must be a floating-point number. Find out why this is so and convert it to floating-point number.

In [8]:
# pd.to_numeric with errors = coerce will convert non-numerical values to NaN.
mpg_data["horsepower"] = pd.to_numeric(mpg_data['horsepower'], errors='coerce')

In [9]:
nose.tools.assert_equal(mpg_data.horsepower.dtype, "float64")

### Problem 4. Missing values: inspection (1 point)
We saw that the `horsepower` column contained null values. Display the rows which contain those values. Assign the resulting dataframe to the `unknown_hp` variable.

In [10]:
def get_unknown_hp(dataframe):
    """
    Returns the rows in the provided dataframe where the "horsepower" column is NaN
    """
    unknown_hp = dataframe[dataframe["horsepower"].isna()]

    return unknown_hp

In [11]:
cars_with_unknown_hp = get_unknown_hp(mpg_data)
print(cars_with_unknown_hp)

      mpg  cylinders  displacement  horsepower  weight  aceleration  \
32   25.0          4          98.0         NaN  2046.0         19.0   
126  21.0          6         200.0         NaN  2875.0         17.0   
330  40.9          4          85.0         NaN  1835.0         17.3   
336  23.6          4         140.0         NaN  2905.0         14.3   
354  34.5          4         100.0         NaN  2320.0         15.8   
374  23.0          4         151.0         NaN  3035.0         20.5   

     model_year  origin                car_name  
32           71       1            "ford pinto"  
126          74       1         "ford maverick"  
330          80       2  "renault lecar deluxe"  
336          80       1    "ford mustang cobra"  
354          81       2           "renault 18i"  
374          82       1        "amc concord dl"  


### Problem 5. Missing data: correction (1 point)
It seems like the `NaN` values are a small fraction of all values. We can try one of several things:
* Remove them
* Replace them (e.g. with the mean power of all cars)
* Look up the models on the internet and try our best guess on the power

The third one is probably the best but the first one will suffice since these records are too few. Remove those values. Save the dataset in the same `mpg_data` variable. Ensure there are no more `NaN`s.

In [12]:
# Clear all data wih NaN values in the dataset
mpg_data = mpg_data.dropna()

In [13]:
nose.tools.assert_equal(len(get_unknown_hp(mpg_data)), 0)

### Problem 6. Years of production (1 + 1 points)
Display all unique model years. Assign them to the variable `model_years`.

In [14]:
def get_unique_model_years(dataframe):
    """
    Returns the unique values of the "model_year" column
    of the dataframe
    """
    model_years = dataframe["model_year"].unique()
    return model_years

In [15]:
model_years = get_unique_model_years(mpg_data)
print(model_years)

[70 71 72 73 74 75 76 77 78 79 80 81 82]


These don't look so good. Convert them to real years, like `70 -> 1970, 71 -> 1971`. Replace the column values in the dataframe.

In [16]:
# Add 1900 to the provided year
def year_to_add(year):
    return 1900 + year

In [17]:
# Convert model year like 70 -> 1970
mpg_data["model_year"] = mpg_data.model_year.apply(year_to_add)

In [18]:
model_years = get_unique_model_years(mpg_data)
print(model_years)

[1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982]


### Problem 7. Exploration: low-power cars (1 point)
The data looks quite good now. Let's try some exploration.

Write a function to find the cars which have the smallest number of cylinders and print their model names. Return a list of car names.

In [19]:
def get_model_names_smallest_cylinders(dataframe):
    """
    Returns the names of the cars with the smallest number of cylinders
    """
    # Create new dataset witl all cars with min cylinders values
    cars_witn_min_cylinders = dataframe[dataframe["cylinders"] == dataframe["cylinders"].min()]
    # Get car names with smallest cylinders
    car_names = cars_witn_min_cylinders["car_name"]
    return car_names

In [20]:
car_names = get_model_names_smallest_cylinders(mpg_data)
print(car_names)
nose.tools.assert_true(car_names.shape == (4,) or car_names.shape == (4, 1))

71     "mazda rx2 coupe"
111          "maxda rx3"
243         "mazda rx-4"
334      "mazda rx-7 gs"
Name: car_name, dtype: object


### Problem 8. Exploration: correlations (1 point)
Finally, let's see some connections between variables. These are also called **correlations**.

Find how to calculate correlations between different columns using `pandas`.

**Hint:** The correlation function in `pandas` returns a `DataFrame` by default. You need only one value from it.

Create a function which accepts a dataframe and two columns and prints the correlation coefficient between those two columns.

In [21]:
def calculate_correlation(dataframe, first_column, second_column):
    """
    Calculates and returns the correlation coefficient between the two columns in the dataframe.
    """
    correlation = dataframe[first_column].corr(dataframe[second_column])
    return correlation

In [22]:
hp_weight = calculate_correlation(mpg_data, "horsepower", "weight")
print("Horsepower:Weight correlation coefficient:", hp_weight)
nose.tools.assert_almost_equal(hp_weight, 0.864537737574, delta = 0.01)


Horsepower:Weight correlation coefficient: 0.8645377375741443


<h1>Quiz Data</h1>

<h3>Null Values</h3>
The "Titanic" dataset (located at <a href="https://raw.githubusercontent.com/Geoyi/Cleaning-Titanic-Data/master/titanic_original.csv"> Titanic</a>) contains information about 887 passengers on the Titanic (out of 2224 total). Which column has the most missing values?

In [23]:
titanic_data = pd.read_csv('https://raw.githubusercontent.com/Geoyi/Cleaning-Titanic-Data/master/titanic_original.csv')
titanic_data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [24]:
titanic_data.isnull().sum()

pclass          1
survived        1
name            1
sex             1
age           264
sibsp           1
parch           1
ticket          1
fare            2
cabin        1015
embarked        3
boat          824
body         1189
home.dest     565
dtype: int64

<h3>Observations and Features</h3>
The Iris dataset is one of the most populat datasets in data processing. You can find the data and description at <a href="https://archive.ics.uci.edu/ml/datasets/iris">Iris</a>. How many observations are there? And how many features? The last column contains the different species of iris plant. How many species are there?

In [25]:
iris_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header = None)
iris_data.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "iris_class"]
iris_data.shape

(150, 5)

In [26]:
len(iris_data["iris_class"].unique())

3

<h3>Iris Data Insights</h3>
Take all Iris flowers whose sepal length is between 4.5 and 10 (inclusive). What is the average petal length of these flowers (in cm)? Round your answer to two decimal places. Write your answer with a decimal point, for example "4.5".

In [27]:
iris_data_sepal = iris_data[iris_data['sepal_length'].between(4.5, 10, inclusive=True)]
iris_data_sepal.shape
round(iris_data_sepal.petal_length.mean(), 2)

3.83

<h3>Wine Quality</h3>
The dataset here: <a href="http://archive.ics.uci.edu/ml/datasets/Wine+Quality">Wine Quality</a> contains information about different red and white wines. Read the two files into Python. Which kinds of wine seem to be preferred (i.e. have greater average quality)? Write "red" or "white" in the textbox.

In [28]:
red_wine_data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")

In [29]:
red_wine_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [30]:
white_wine_data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=";")

In [31]:
white_wine_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [32]:
red_wine_data["quality"].describe()["mean"]

5.6360225140712945

In [33]:
white_wine_data["quality"].describe()["mean"]

5.87790935075541

<h3>Quality and Alcohol Content</h3>
Does higher alcohol content mean better quality? We can't say directly, but we might explore the data. What is the correlation of alcohol content to quality for red wines? Round your answer to two decimal places. 
What is the correlation of alcohol content to quality for white wines? Round your answer to two decimal places.

In [34]:
red_wine_correlation = round(red_wine_data["alcohol"].corr(red_wine_data["quality"]), 2)
white_wine_correlation = round(white_wine_data["alcohol"].corr(white_wine_data["quality"]), 2)

In [35]:
red_wine_correlation, white_wine_correlation

(0.48, 0.44)